In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import json
import optuna
from functools import partial
from joblib import Memory

In [2]:
def load_clustering_info():
    """Load clustering information from CSV file"""
    cluster_file = "/kaggle/input/ai-data4clustering-dict/fuzzy_clusters.csv"
    try:
        clusters_df = pd.read_csv(cluster_file)
        print(f" Loaded clustering info: {clusters_df.shape[0]} rows")
        # Check if required columns exist
        required_cols = ['ROW', 'COL', 'CLUSTER']
        if not all(col in clusters_df.columns for col in required_cols):
            print(f"[ERRORS] Clustering file missing required columns: {required_cols}")
            return None
        return clusters_df
    except Exception as e:
        print(f"[ERRORS] Error loading clustering file: {str(e)}")
        return None

In [3]:


# Setup memory cache
cache_dir = './joblib_cache'
memory = Memory(cache_dir, verbose=0)

class Config:
    TARGET = 'AWS'
    USE_LAG_FEATURES = True
    USE_ROLLING_STATISTICS = True

# Paths
base_path = "/kaggle/input/ai-dataimputedataset-k-fold"
months = ["2019-04", "2019-10", "2020-04", "2020-10"]
folds = [f"fold_{i}" for i in range(1, 6)]

# Define selected features
selected_features = [
    'TCW', 'TCLW', 'R250', 'R500', 'R850', 'U850', 'V850', 'EWSS', 'KX', 'CAPE', 'SSHF', 'PEV'
]

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model parameters - now configurable via Optuna
MODEL_CONFIG = {
    "04": {"BATCH_SIZE": 64, "EPOCHS": 10},
    "10": {"BATCH_SIZE": 128, "EPOCHS": 15},
}

# Create lag features
@memory.cache
def create_lag_features(train_df, test_df, target_column, lag_steps, groupby_cols):
    """Create lag features for the target column"""
    result_df = test_df.copy()
    
    # Combine train and test for continuous time series
    combined_df = pd.concat([train_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
    
    # Create lag features
    for lag in lag_steps:
        combined_df[f'{target_column}_lag{lag}'] = combined_df.groupby(groupby_cols)[target_column].shift(lag)
    
    # Extract only the test portion with lag features
    result_df = combined_df.iloc[len(train_df):].reset_index(drop=True)
    return result_df

# Create rolling statistics
@memory.cache
def create_rolling_statistics(train_df, test_df, target_column, window_sizes, groupby_cols):
    """Create rolling statistics features for the target column"""
    result_df = test_df.copy()
    
    # Combine train and test for continuous rolling stats
    combined_df = pd.concat([train_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
    
    # Create rolling features
    for window in window_sizes:
        # Rolling mean
        combined_df[f'{target_column}_rollmean_{window}'] = combined_df.groupby(groupby_cols)[target_column].transform(
            lambda x: x.rolling(window, min_periods=1).mean())
        # Rolling std
        combined_df[f'{target_column}_rollstd_{window}'] = combined_df.groupby(groupby_cols)[target_column].transform(
            lambda x: x.rolling(window, min_periods=1).std())
    
    # Extract only the test portion with rolling features
    result_df = combined_df.iloc[len(train_df):].reset_index(drop=True)
    return result_df

# Handle missing values
@memory.cache
def handle_missing_values(df, lag_steps, window_sizes):
    """Handle missing values in the dataframe"""
    result_df = df.copy()
    
    # Fill NaN values in lag features with 0
    for lag in lag_steps:
        lag_col = f'{Config.TARGET}_lag{lag}'
        if lag_col in result_df.columns:
            result_df[lag_col] = result_df[lag_col].fillna(0)
    
    # Fill NaN values in rolling features with 0
    for window in window_sizes:
        mean_col = f'{Config.TARGET}_rollmean_{window}'
        std_col = f'{Config.TARGET}_rollstd_{window}'
        
        if mean_col in result_df.columns:
            result_df[mean_col] = result_df[mean_col].fillna(0)
        
        if std_col in result_df.columns:
            result_df[std_col] = result_df[std_col].fillna(0)
    
    # Fill remaining NaNs with 0
    result_df = result_df.fillna(0)
    
    return result_df



# Modified load_and_process_data function to filter features
@memory.cache
@memory.cache
def load_and_process_data(file_path, train_file_path=None, lag_steps=None, window_sizes=None, cluster_df=None, cluster_id=None):
    """
    Load and process data from file_path with optional lag features and rolling statistics
    If train_file_path is provided, use it for creating lag and rolling features
    If cluster_df and cluster_id are provided, filter data by cluster
    """
    
    try:
        raw_df = pd.read_csv(file_path)
        
        # Filter by cluster if specified
        if cluster_df is not None and cluster_id is not None:
            # Get ROW, COL pairs in this cluster
            cluster_points = cluster_df[cluster_df['CLUSTER'] == cluster_id][['ROW', 'COL']].values.tolist()
            
            # Filter raw_df to keep only points in this cluster
            raw_df = raw_df[raw_df.apply(lambda row: [row['ROW'], row['COL']] in cluster_points, axis=1)]
            
            if raw_df.empty:
                print(f" No data points found for cluster {cluster_id}")
                return pd.DataFrame()
        
        # Filter to keep only selected features plus essential columns
        essential_cols = ['DATETIME', 'ROW', 'COL', Config.TARGET]
        feature_cols = [col for col in selected_features if col in raw_df.columns]
        filtered_cols = essential_cols + feature_cols
        
        # Keep only needed columns
        raw_df = raw_df[filtered_cols]
        
        # Convert target to numeric and handle NaN/inf values
        raw_df[Config.TARGET] = pd.to_numeric(raw_df[Config.TARGET], errors='coerce')
        raw_df = raw_df.replace([np.inf, -np.inf], np.nan)
        df = raw_df.dropna(subset=[Config.TARGET]).copy()
        
        # Sort by datetime for proper sequence handling
        if 'DATETIME' in df.columns:
            df = df.sort_values("DATETIME").reset_index(drop=True)
        
        # If we're in train mode or not creating lag/rolling features
        if train_file_path is None or (lag_steps is None and window_sizes is None):
            return df
        
        # Otherwise, we're in eval mode and need to carefully create features
        train_df = pd.read_csv(train_file_path)
        
        # Filter training data by cluster if specified
        if cluster_df is not None and cluster_id is not None:
            cluster_points = cluster_df[cluster_df['CLUSTER'] == cluster_id][['ROW', 'COL']].values.tolist()
            train_df = train_df[train_df.apply(lambda row: [row['ROW'], row['COL']] in cluster_points, axis=1)]
            
            if train_df.empty:
                print(f" No training data points found for cluster {cluster_id}")
                return pd.DataFrame()
        
        # Filter training data to keep only selected features
        train_df = train_df[filtered_cols]
        
        train_df[Config.TARGET] = pd.to_numeric(train_df[Config.TARGET], errors='coerce')
        train_df = train_df.replace([np.inf, -np.inf], np.nan)
        train_df = train_df.dropna(subset=[Config.TARGET]).copy()
        
        if 'DATETIME' in train_df.columns:
            train_df = train_df.sort_values("DATETIME").reset_index(drop=True)
        
        # Create lag features if needed
        if Config.USE_LAG_FEATURES and lag_steps:
            df = create_lag_features(train_df, df, Config.TARGET, lag_steps, ['ROW', 'COL'])
        
        # Create rolling statistics if needed
        if Config.USE_ROLLING_STATISTICS and window_sizes:
            df = create_rolling_statistics(train_df, df, Config.TARGET, window_sizes, ['ROW', 'COL'])
        
        # Handle missing values
        if lag_steps or window_sizes:
            df = handle_missing_values(df, lag_steps or [], window_sizes or [])
        return df
        
    except Exception as e:
        print(f"[ERRORS] Error loading or processing data: {str(e)}")
        return pd.DataFrame()

# Enhanced LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.0, time_step_out=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, time_step_out)
        self.time_step_out = time_step_out

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# Create sequences with configurable input and output time steps
def create_sequences(df, input_cols, target_col, time_step_in, time_step_out=1, stride=1):
    """
    Create sequences from dataframe with configurable input and output time steps
    - time_step_in: number of time steps for input
    - time_step_out: number of future steps to predict
    - stride: step size for sliding window
    """
    sequences, targets = [], []
    grouped = df.groupby(['ROW', 'COL'])
    
    for _, group in grouped:
        # Make sure group is sorted by time
        if 'DATETIME' in group.columns:
            group = group.sort_values("DATETIME")
            
        data = group[input_cols].values
        target_data = group[target_col].values
        
        if len(data) < time_step_in + time_step_out:
            continue
        
        for i in range(0, len(data) - time_step_in - time_step_out + 1, stride):
            seq = data[i:i+time_step_in]
            if time_step_out == 1:
                target = target_data[i+time_step_in]
                targets.append(target)
            else:
                target = target_data[i+time_step_in:i+time_step_in+time_step_out]
                targets.append(target)
            sequences.append(seq)
            
    if not sequences:
        return torch.tensor([]), torch.tensor([])
    
    if time_step_out == 1:
        return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).unsqueeze(1)
    else:
        return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

# Prepare data loaders
# Modified prepare_data_loaders function
def prepare_data_loaders(month, fold, lag_steps, window_sizes, time_step_in, time_step_out, batch_size, stride=1, cluster_df=None, cluster_id=None):
    """Prepare data loaders with specific time steps and features for a specific cluster"""
    folder = os.path.join(base_path, month, fold)
    
    # Load train data
    train_df = load_and_process_data(
        os.path.join(folder, "processed_train.csv"), 
        cluster_df=cluster_df, 
        cluster_id=cluster_id
    )
    
    # Load validation data
    val_df = load_and_process_data(
        os.path.join(folder, "processed_val.csv"), 
        cluster_df=cluster_df, 
        cluster_id=cluster_id
    )
    
    # Load test data
    test_df = load_and_process_data(
        os.path.join(folder, "merged_test.csv"), 
        cluster_df=cluster_df, 
        cluster_id=cluster_id
    )
    
    if train_df.empty or val_df.empty or test_df.empty:
        print(f" One or more datasets are empty for cluster {cluster_id}")
        return None, None, None, 0
    
    # Rest of the function remains the same as your original code...
    # Sort data by time
    if 'DATETIME' in train_df.columns:
        train_df = train_df.sort_values("DATETIME").reset_index(drop=True)
    if 'DATETIME' in val_df.columns:
        val_df = val_df.sort_values("DATETIME").reset_index(drop=True)
    if 'DATETIME' in test_df.columns:
        test_df = test_df.sort_values("DATETIME").reset_index(drop=True)
    
    # Create features before further processing
    # 1. Create lag features separately for each dataset
    if Config.USE_LAG_FEATURES and lag_steps:
        # Create lag features for train set using itself
        for lag in lag_steps:
            train_df[f'{Config.TARGET}_lag{lag}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        
        # Create lag features for validation set using train + val
        train_val_df = pd.concat([train_df, val_df]).sort_values('DATETIME').reset_index(drop=True)
        for lag in lag_steps:
            train_val_df[f'{Config.TARGET}_lag{lag}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        val_df = train_val_df.iloc[len(train_df):].reset_index(drop=True)
        
        # Create lag features for test set using train + val + test
        full_df = pd.concat([train_df, val_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
        for lag in lag_steps:
            full_df[f'{Config.TARGET}_lag{lag}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].shift(lag)
        test_df = full_df.iloc[len(train_df) + len(val_df):].reset_index(drop=True)
    
    # 2. Create rolling statistics separately for each dataset
    if Config.USE_ROLLING_STATISTICS and window_sizes:
        # Create rolling stats for train set
        for window in window_sizes:
            train_df[f'{Config.TARGET}_rollmean_{window}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            train_df[f'{Config.TARGET}_rollstd_{window}'] = train_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        
        # Create rolling stats for validation set
        train_val_df = pd.concat([train_df, val_df]).sort_values('DATETIME').reset_index(drop=True)
        for window in window_sizes:
            train_val_df[f'{Config.TARGET}_rollmean_{window}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            train_val_df[f'{Config.TARGET}_rollstd_{window}'] = train_val_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        val_df = train_val_df.iloc[len(train_df):].reset_index(drop=True)
        
        # Create rolling stats for test set
        full_df = pd.concat([train_df, val_df, test_df]).sort_values('DATETIME').reset_index(drop=True)
        for window in window_sizes:
            full_df[f'{Config.TARGET}_rollmean_{window}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).mean())
            full_df[f'{Config.TARGET}_rollstd_{window}'] = full_df.groupby(['ROW', 'COL'])[Config.TARGET].transform(
                lambda x: x.rolling(window, min_periods=1).std())
        test_df = full_df.iloc[len(train_df) + len(val_df):].reset_index(drop=True)
    
    # Handle missing values
    train_df = train_df.fillna(0)
    val_df = val_df.fillna(0)
    test_df = test_df.fillna(0)
    
    # Prepare feature columns - exclude DATETIME, ROW, COL and target column
    # Include selected_features that are in the DataFrame
    basic_cols = [col for col in selected_features if col in train_df.columns]
    lag_cols = [f'{Config.TARGET}_lag{lag}' for lag in lag_steps if f'{Config.TARGET}_lag{lag}' in train_df.columns]
    roll_cols = []
    
    for window in window_sizes:
        mean_col = f'{Config.TARGET}_rollmean_{window}'
        std_col = f'{Config.TARGET}_rollstd_{window}'
        if mean_col in train_df.columns:
            roll_cols.append(mean_col)
        if std_col in train_df.columns:
            roll_cols.append(std_col)
    
    feature_cols = basic_cols + lag_cols + roll_cols
    
    # Check if we have any features
    if not feature_cols:
        print(" No features detected! Creating default lag feature.")
        # Create at least one default lag feature
        default_lag = 1
        for df in [train_df, val_df, test_df]:
            df[f'{Config.TARGET}_lag{default_lag}'] = df.groupby(['ROW', 'COL'])[Config.TARGET].shift(default_lag)
            df = df.fillna(0)
        
        feature_cols = [f'{Config.TARGET}_lag{default_lag}']
    
    # Create sequences
    train_x, train_y = create_sequences(train_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    val_x, val_y = create_sequences(val_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    test_x, test_y = create_sequences(test_df, feature_cols, Config.TARGET, time_step_in, time_step_out, stride)
    
    if train_x.numel() == 0 or val_x.numel() == 0 or test_x.numel() == 0:
        print(f" One or more sequence sets are empty for cluster {cluster_id}")
        return None, None, None, 0
    
    # Create data loaders
    train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(val_x, val_y), batch_size=batch_size)
    test_loader = DataLoader(TensorDataset(test_x, test_y), batch_size=batch_size)
    
    return train_loader, val_loader, test_loader, train_x.shape[2]

# Evaluate model
def evaluate_model(model, loader):
    """Evaluate model and return metrics"""
    model.eval()
    preds, targets = [], []
    
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            pred = model(xb).cpu()
            preds.append(pred)
            targets.append(yb)
    
    preds = torch.cat(preds).squeeze().numpy()
    targets = torch.cat(targets).squeeze().numpy()
    
    # Handle multi-step output
    if len(preds.shape) > 1 and preds.shape[1] > 1:
        # For multi-step evaluation, we'll calculate metrics on the last predicted step
        preds_last = preds[:, -1]
        targets_last = targets[:, -1] if len(targets.shape) > 1 else targets
        
        rmse = mean_squared_error(targets_last, preds_last, squared=False)
        bias = np.mean(preds_last - targets_last)
        r = r2_score(targets_last, preds_last)
        csi = np.sum((preds_last > 0.1) & (targets_last > 0.1)) / (np.sum((preds_last > 0.1) | (targets_last > 0.1)) + 1e-9)
    else:
        rmse = mean_squared_error(targets, preds, squared=False)
        bias = np.mean(preds - targets)
        r = r2_score(targets, preds)
        csi = np.sum((preds > 0.1) & (targets > 0.1)) / (np.sum((preds > 0.1) | (targets > 0.1)) + 1e-9)
    
    return rmse, bias, r, csi

In [4]:
def run_cluster_evaluation_from_checkpoint():
    """Run evaluation on clusters using pre-trained models from checkpoints"""
    # Load clustering information
    cluster_df = load_clustering_info()
    if cluster_df is None:
        print("[ERRORS] Failed to load clustering information")
        return
    
    # Get unique cluster IDs
    cluster_ids = cluster_df['CLUSTER'].unique()
    
    # Results storage
    all_results = []
    best_month_results = []  # Store best fold results for each month
    
    for month in months:
        print(f"\n###### Processing month: {month}")
        
        # Load best parameters for this month
        best_params_path = f"/kaggle/input/lstm-checkpoint/best_params_{month}.json"
        try:
            with open(best_params_path, "r") as f:
                best_params = json.load(f)
            print(f" Loaded best parameters from {best_params_path}")
        except Exception as e:
            print(f"[ERRORS] Error loading best parameters: {str(e)}")
            continue
        
        # Store results for all folds of this month
        month_fold_results = []
        
        # Process each fold for this month
        for fold in folds:
            print(f"\n##### Processing {month}, {fold}")
            
            # Process each cluster
            cluster_results = []
            for cluster_id in cluster_ids:
                print(f"\n##### Processing cluster {cluster_id}")
                # Use our evaluation function
                result = evaluate_cluster_with_checkpoint(month, fold, best_params, cluster_df, cluster_id)
                if result:
                    cluster_results.append(result)
                    all_results.append(result)
            
            # Calculate mean performance across clusters for this fold
            if cluster_results:
                mean_rmse = sum(r["test_rmse"] for r in cluster_results) / len(cluster_results)
                mean_bias = sum(r["test_bias"] for r in cluster_results) / len(cluster_results)
                mean_r = sum(r["test_r"] for r in cluster_results) / len(cluster_results)
                mean_csi = sum(r["test_csi"] for r in cluster_results) / len(cluster_results)
                
                # Create fold summary
                fold_summary = {
                    "month": month,
                    "fold": fold,
                    "mean_test_rmse": round(mean_rmse, 4),
                    "mean_test_bias": round(mean_bias, 4),
                    "mean_test_r": round(mean_r, 4),
                    "mean_test_csi": round(mean_csi, 4),
                    "num_clusters": len(cluster_results)
                }
                
                # Add to month's fold results
                month_fold_results.append(fold_summary)
                
                print(f"\n Mean test performance across clusters for {month}, {fold}:")
                print(f"  RMSE: {mean_rmse:.4f}, Bias: {mean_bias:.4f}, R²: {mean_r:.4f}, CSI: {mean_csi:.4f}")
                
                # Save summary for this fold
                with open(f"cluster_summary_{month}_{fold}.json", "w") as f:
                    json.dump(fold_summary, f, indent=2)
        
        # Find best fold for this month (lowest RMSE)
        if month_fold_results:
            best_fold_result = min(month_fold_results, key=lambda x: x["mean_test_rmse"])
            best_month_results.append(best_fold_result)
            
            print(f"\n Best fold for {month}: {best_fold_result['fold']}")
            print(f"  RMSE: {best_fold_result['mean_test_rmse']:.4f}, Bias: {best_fold_result['mean_test_bias']:.4f}, R²: {best_fold_result['mean_test_r']:.4f}, CSI: {best_fold_result['mean_test_csi']:.4f}")
    
    # Create a dataframe with all results
    if all_results:
        all_results_df = pd.DataFrame([
            {
                "month": r["month"],
                "fold": r["fold"],
                "cluster": r["cluster"],
                "test_rmse": r["test_rmse"],
                "test_bias": r["test_bias"],
                "test_r": r["test_r"],
                "test_csi": r["test_csi"],
                "val_rmse": r["val_rmse"],
                "val_bias": r["val_bias"],
                "val_r": r["val_r"],
                "val_csi": r["val_csi"]
            } for r in all_results
        ])
        
        # Save all results
        all_results_df.to_csv("lstm_cluster_results.csv", index=False)
        print("\n All cluster results saved to lstm_cluster_results.csv")
        
        # Create a dataframe with best fold results per month
        best_month_df = pd.DataFrame(best_month_results)
        best_month_df.to_csv("lstm_best_fold_per_month.csv", index=False)
        print(" Best fold results per month saved to lstm_best_fold_per_month.csv")
        
        # Calculate overall mean performance across best folds of all months
        if best_month_results:
            overall_mean_rmse = sum(r["mean_test_rmse"] for r in best_month_results) / len(best_month_results)
            overall_mean_bias = sum(r["mean_test_bias"] for r in best_month_results) / len(best_month_results)
            overall_mean_r = sum(r["mean_test_r"] for r in best_month_results) / len(best_month_results)
            overall_mean_csi = sum(r["mean_test_csi"] for r in best_month_results) / len(best_month_results)
            
            overall_results = {
                "overall_mean_rmse": round(overall_mean_rmse, 4),
                "overall_mean_bias": round(overall_mean_bias, 4),
                "overall_mean_r": round(overall_mean_r, 4),
                "overall_mean_csi": round(overall_mean_csi, 4),
                "num_months": len(best_month_results)
            }
            
            # Save overall results
            with open("lstm_overall_results.json", "w") as f:
                json.dump(overall_results, f, indent=2)
            
            print("\n Overall mean performance across best folds of all months:")
            print(f"  RMSE: {overall_mean_rmse:.4f}, Bias: {overall_mean_bias:.4f}, R²: {overall_mean_r:.4f}, CSI: {overall_mean_csi:.4f}")
            
            # Display best fold results for each month
            print("\n Best Fold Results by Month:")
            print(best_month_df)

In [5]:
def evaluate_cluster_with_checkpoint(month, fold, best_params, cluster_df=None, cluster_id=None):
    """Evaluate a pre-trained model on a specific cluster without retraining"""
    print(f"\n Evaluating cluster {cluster_id} for {month}, {fold} using checkpoint")
    
    # Extract parameters for data preparation
    time_step_in = best_params["time_step_in"]
    time_step_out = best_params["time_step_out"]
    stride = best_params["stride"]
    
    # Extract lag steps and window sizes
    num_lags = best_params["num_lags"]
    lag_steps = [best_params[f"lag_{i}"] for i in range(num_lags)]
    
    num_windows = best_params["num_windows"]
    window_sizes = [best_params[f"window_{i}"] for i in range(num_windows)]
    
    # Get config
    config_key = month.split("-")[1]
    config = MODEL_CONFIG[config_key]
    batch_size = config["BATCH_SIZE"]
    
    # Prepare data for this cluster only
    _, val_loader, test_loader, input_size = prepare_data_loaders(
        month, fold, lag_steps, window_sizes, time_step_in, time_step_out, batch_size, stride,
        cluster_df=cluster_df, cluster_id=cluster_id
    )
    
    if val_loader is None or test_loader is None:
        print(f"[ERRORS] Failed to prepare data for cluster {cluster_id}")
        return None
    
    # Load the pre-trained model for this month
    checkpoint_path = f"/kaggle/input/lstm-checkpoint/best_model_{month}.pt"
    
    # Create model with the same architecture as the pre-trained one
    model = LSTMModel(
        input_size=input_size,
        hidden_size=best_params["hidden_size"],
        num_layers=best_params["num_layers"],
        dropout=best_params["dropout"],
        time_step_out=time_step_out
    ).to(DEVICE)
    
    try:
        # Load checkpoint state
        model.load_state_dict(torch.load(checkpoint_path))
        print(f" Successfully loaded checkpoint from {checkpoint_path}")
    except Exception as e:
        print(f"[ERRORS] Error loading checkpoint: {str(e)}")
        return None
    
    # Set model to evaluation mode
    model.eval()
    
    # Evaluate on validation set
    print(f" Evaluating on validation set for cluster {cluster_id}...")
    val_rmse, val_bias, val_r, val_csi = evaluate_model(model, val_loader)
    print(f"  Val RMSE: {val_rmse:.4f}, Bias: {val_bias:.4f}, R²: {val_r:.4f}, CSI: {val_csi:.4f}")
    
    # Evaluate on test set
    print(f" Evaluating on test set for cluster {cluster_id}...")
    test_rmse, test_bias, test_r, test_csi = evaluate_model(model, test_loader)
    print(f"  Test RMSE: {test_rmse:.4f}, Bias: {test_bias:.4f}, R²: {test_r:.4f}, CSI: {test_csi:.4f}")
    
    return {
        "month": month,
        "fold": fold,
        "cluster": cluster_id,
        "test_rmse": round(test_rmse, 4),
        "test_bias": round(test_bias, 4),
        "test_r": round(test_r, 4),
        "test_csi": round(test_csi, 4),
        "val_rmse": round(val_rmse, 4),
        "val_bias": round(val_bias, 4),
        "val_r": round(val_r, 4),
        "val_csi": round(val_csi, 4)
    }

In [6]:
run_cluster_evaluation_from_checkpoint()

 Loaded clustering info: 334 rows

###### Processing month: 2019-04
 Loaded best parameters from /kaggle/input/lstm-checkpoint/best_params_2019-04.json

##### Processing 2019-04, fold_1

##### Processing cluster 0

 Evaluating cluster 0 for 2019-04, fold_1 using checkpoint


<ipython-input-3-62cb43d7e5fe>:233: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(sequences, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)
<ipython-input-5-ca89ff15378e>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unl

 Successfully loaded checkpoint from /kaggle/input/lstm-checkpoint/best_model_2019-04.pt
 Evaluating on validation set for cluster 0...
  Val RMSE: 0.0648, Bias: 0.0335, R²: -0.3288, CSI: 0.0000
 Evaluating on test set for cluster 0...
  Test RMSE: 0.9646, Bias: -0.1126, R²: -0.0111, CSI: 0.0991

##### Processing cluster 2

 Evaluating cluster 2 for 2019-04, fold_1 using checkpoint


<ipython-input-5-ca89ff15378e>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


 Successfully loaded checkpoint from /kaggle/input/lstm-checkpoint/best_model_2019-04.pt
 Evaluating on validation set for cluster 2...
  Val RMSE: 0.0569, Bias: 0.0381, R²: -0.8003, CSI: 0.0000
 Evaluating on test set for cluster 2...
  Test RMSE: 1.0960, Bias: -0.1579, R²: -0.0099, CSI: 0.0901

##### Processing cluster 1

 Evaluating cluster 1 for 2019-04, fold_1 using checkpoint


...

<ipython-input-5-ca89ff15378e>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


 Successfully loaded checkpoint from /kaggle/input/lstm-checkpoint/best_model_2020-10.pt
 Evaluating on validation set for cluster 2...
  Val RMSE: 0.2312, Bias: 0.0583, R²: 0.5093, CSI: 0.4409
 Evaluating on test set for cluster 2...
  Test RMSE: 0.2137, Bias: 0.0783, R²: -0.1077, CSI: 0.3783

##### Processing cluster 1

 Evaluating cluster 1 for 2020-10, fold_5 using checkpoint


<ipython-input-5-ca89ff15378e>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


 Successfully loaded checkpoint from /kaggle/input/lstm-checkpoint/best_model_2020-10.pt
 Evaluating on validation set for cluster 1...
  Val RMSE: 0.5491, Bias: 0.0713, R²: 0.7199, CSI: 0.6839
 Evaluating on test set for cluster 1...
  Test RMSE: 0.1953, Bias: 0.0613, R²: 0.8744, CSI: 0.7149

##### Processing cluster 3

 Evaluating cluster 3 for 2020-10, fold_5 using checkpoint
 Successfully loaded checkpoint from /kaggle/input/lstm-checkpoint/best_model_2020-10.pt
 Evaluating on validation set for cluster 3...
  Val RMSE: 1.7074, Bias: -0.3005, R²: 0.6417, CSI: 0.8255
 Evaluating on test set for cluster 3...
  Test RMSE: 0.6467, Bias: 0.0535, R²: 0.7158, CSI: 0.7916

 Mean test performance across clusters for 2020-10, fold_5:
  RMSE: 0.3068, Bias: 0.0669, R²: 0.5795, CSI: 0.6322

 Best fold for 2020-10: fold_5
  RMSE: 0.3068, Bias: 0.0669, R²: 0.5794, CSI: 0.6322

 All cluster results saved to lstm_cluster_results.csv
 Best fold results per month saved to lstm_best_fold_per_month.csv

<ipython-input-5-ca89ff15378e>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))
